In [2]:
%cd drive/My Drive/CS5814/HW3


/content/drive/My Drive/CS5814/HW3


In [45]:
pip install transformers


In [46]:
pip install xlsxwriter


     |████████████████████████████████| 149 kB 4.1 MB/s 


In [58]:
import json
import os
import random
import gc
import xlsxwriter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
import pickle

import torch.nn as nn

torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)


cuda:0


In [5]:
def preprocess_data(file_loc='code_dataset.jsonl', generate_histogram=False):
    """
    Loads and processing the jsonl file,

    :param file_loc: location of target jsonl file
    :param generate_histogram: Flag to display histogram of function lengths
    :return: dataframe of preprocessed jsons
    """

    with open(file_loc, 'r') as json_file:
        json_list = list(json_file)

    code_list = []
    for json_str in json_list:
        result = json.loads(json_str)
        code_list.append(result)

    code_df = pd.DataFrame(code_list)

    total = code_df['target'].sum()
    proportion = total / code_df.shape[0]

    print("Insecure code counts: {}, Total code counts: {}, Proportion {}".format(total, code_df.shape[0], proportion))

    if generate_histogram:
        plt.hist(code_df['func'].str.len(), bins=100)
        plt.show()

    return code_df

In [6]:


def split_data(input_data, attention_data, label_data, train_ratio=0.8, val_ratio=0.10, max_len=512):
    """
    Splits data in accordance with provdied ratios, additionally discards functions with > max_len tokens
        as these will not be processed by the model will (can truncate, yet may truncate the error in the code)

    :param input_data: input functions
    :param attention_data: attention map
    :param label_data: target labels
    :param train_ratio: ratio of data to train on
    :param val_ratio: ratio of data to validate with (test is inferred from this and train)
    :param max_len: max number of tokens allowed for training date

    :return: 3 tuples for train val and test containing (input, attention, target)
    """
    # Removing excessively long elements from dataset
    valid_token_index = [i for i in range(len(input_data)) if len(input_data[i]) <= max_len]
    X_data = np.array(input_data)[valid_token_index]
    A_data = np.array(attention_data)[valid_token_index]
    Y_data = np.array(label_data)[valid_token_index]

    dataset_size = len(X_data)

    # Determining index to split dataset
    random_id = random.sample(range(dataset_size), dataset_size)
    train_split_id = int(train_ratio * dataset_size)
    val_split_id = int((train_ratio + val_ratio) * dataset_size)

    train_ids = random_id[:train_split_id]
    val_ids = random_id[train_split_id:val_split_id]
    test_ids = random_id[val_split_id:]

    X_train = torch.tensor(list(X_data[train_ids]))
    A_train = torch.tensor(list(A_data[train_ids]))
    Y_train = torch.tensor(list(Y_data[train_ids]))

    X_val = torch.tensor(list(X_data[val_ids]))
    A_val = torch.tensor(list(A_data[val_ids]))
    Y_val = torch.tensor(list(Y_data[val_ids]))

    X_test = torch.tensor(list(X_data[test_ids]))
    A_test = torch.tensor(list(A_data[test_ids]))
    Y_test = torch.tensor(list(Y_data[test_ids]))

    return (X_train, A_train, Y_train), (X_val, A_val, Y_val), (X_test, A_test, Y_test)



In [75]:

def tokenize(code_df, model_name='codebert-base'):
    """
    Apply the tokenizer from the huggingface pretrained model

    :param code_df: dataframe of preprocess code (from jsonl)
    :param model_name: model name (targeting local install)
    :return: 3 tuples for train val and test containing (input, attention, target)
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    inputs = tokenizer(code_df['func'].tolist(), truncation=False, padding='max_length', max_length=512)

    input_data = inputs['input_ids']
    attention_data = inputs['attention_mask']
    label_data = torch.tensor(code_df['target'].tolist())  # TODO - this can be directly converted to a np array

    print("Data points: {}".format(len(input_data)))

    return split_data(input_data, attention_data, label_data, max_len=512)



In [15]:
def tokenize_256(code_df, model_name='codebert-base'):
    """
    Apply the tokenizer from the huggingface pretrained model

    :param code_df: dataframe of preprocess code (from jsonl)
    :param model_name: model name (targeting local install)
    :return: 3 tuples for train val and test containing (input, attention, target)
    """
    tokenizer = RobertaTokenizer.from_pretrained(model_name)

    inputs = tokenizer.batch_encode_plus(code_df['func'].tolist(), truncation=False)

    input_data = inputs['input_ids']
    attention_data = inputs['attention_mask']
    label_data = torch.tensor(code_df['target'].tolist())  # TODO - this can be directly converted to a np array

    pruned_data = list(np.array(code_df['func'])[[i for i in range(len(input_data)) if len(input_data[i]) <= 256]])

    inputs = tokenizer.batch_encode_plus(pruned_data, truncation=False, padding='max_length', max_length=256)
    input_data = inputs['input_ids']
    attention_data = inputs['attention_mask']
    label_data = torch.tensor(code_df['target'].tolist())  # TODO - this can be directly converted to a np array


    return split_data(input_data, attention_data, label_data, max_len=512)


In [164]:

def train(model, train_data, val_data, epochs=5, batch_size=16, learning_rate=2e-5, validate_per=500,
          run_name="temp", run_descrption=None):
    """
    Main fine-tuning training loop for the provided model

    :param model: model loaded with predefined weights
    :param train_data: tuple of X_train, A_train, Y_train (X = inputs, A = attention, Y = target)
    :param val_data: tuple X_val, A_val, Y_val
    :param epochs: number of epochs for training
    :param batch_size: batch size (see note below about batch_hack)
    :param learning_rate: optimizer learning rate
    :param validate_per: number of weight updates before validation occurs
                            (notes: - if batch_size = 32, and validate_per = 32, validation will occur every batch
                                    - this is wrt the start of each epoch
                                    - validation will always occour at the start of each epoch (step 0))
    :param run_name: name used to saving checkpoints and log files within codebert_finetune_runs
    :param run_descrption: string that is saved to info.txt describing the run


    :return: None (models are saved in checkpoints along with log data)
    """


    # Saving run description.txt
    if run_descrption is not None:
        with open("{}/info.txt".format(dir_name), "a+") as f:
            f.write(run_descrption)

    # Unpacking data
    X_train, A_train, Y_train = train_data
    X_val, A_val, Y_val = val_data


    batch_hack = batch_size  # See note below regarding limited GPU memory

    # Initializing arrays for tracking loss
    train_loss_hist = []
    val_loss_hist = []
    train_pred_hist = []
    # Counter to track batches (see note below related to GPU memory)
    batch_count = 0
    # validate_per_batch = int(validate_per/batch_hack)

    # Moving model to GPU if configured
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    validate_per = int(validate_per/batch_size)

    for epoch in range(epochs):

        # Generating random index for manual shuffling of data each epoch as note using DataLoaders
        permutation = torch.randperm(X_train.shape[0])

        # Note here that only a single element is loaded at each iteration (batch size = 1) due to GPU memory constraint
        for batch_id, i in enumerate(range(0, X_train.shape[0], batch_hack)):

            # Loading batch and moving to device
            indices = permutation[i:i + batch_hack]
            batch_X, batch_Y, batch_A = X_train[indices].to(device), Y_train[indices].to(device), A_train[indices].to(device)

            model.train()

            # Forward pass
            outputs = model(batch_X,labels=batch_Y, attention_mask=batch_A)
            #loss = criterion(loss_clsf.float(), batch_Y_one_hot.float())
            loss = outputs.loss
            #print(loss)

            # Backward pass
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_clsf = nn.Softmax(dim=1)(outputs.logits)
            acc = np.average(torch.eq(batch_Y.cpu(), loss_clsf.argmax(axis=1).cpu()))
            #rint(correct)

            # Tracking loss
            train_loss_hist.append(float(loss.item()))
            train_pred_hist.append(acc)

            # Training output
            train_output = "Epoch:{} Step:{} Training_loss:{:.6f}, Acc_avg:{:.2f}%".format(epoch, i, loss.item(), np.sum(100*train_pred_hist[-50:])/min(len(train_pred_hist), 50))
            print(train_output+" Training_loss_avg:{:.6f}".format(np.average(train_loss_hist[-50:])))
            with open("{}/train_loss.txt".format(dir_name), "a+") as f:
                f.write(train_output+"\n")

            # Validation
            if batch_id % validate_per == 0:
                val_loss_total = 0
                model.eval()
                print("Validating:")
                val_acc = []
                for val_badtch_id, j in tqdm(enumerate(range(0, X_val.shape[0], batch_hack))):
                    # Loading singular validation data (overwrites train data as can only load 1 intp GPU)
                    batch_X, batch_Y, batch_A = X_val[j:j+batch_hack].to(device), Y_val[j:j+batch_hack].to(device), A_val[j:j+batch_hack].to(device)

                    with torch.no_grad():
                        val_outputs = model(batch_X, labels=batch_Y, attention_mask=batch_A)
                    val_loss_total += float(val_outputs['loss'].item())

                    
                    val_clsf = nn.Softmax(dim=1)(val_outputs.logits)
                    val_acc.append(np.average(torch.eq(batch_Y.cpu(), val_clsf.argmax(axis=1).cpu())))

                    del batch_X
                    del batch_Y

                # Adding average loss to tracker
                val_average = val_loss_total / (val_badtch_id+1)
                val_loss_hist.append(val_average)

                # Validation output and logging
                val_output = "Epoch:{} Step:{} Val_loss:{:.6f}, Val_Acc_avg:{:.2f}%".format(epoch, i, val_average, np.sum(100*val_acc[-50:])/min(len(val_acc), 50))
                print(val_output)
                with open("{}/val_los.txt".format(dir_name), "a+") as f:
                    f.write(val_output+"\n")

        # End of epoch checkpoint
        model.save_pretrained("{}/epoch_{}".format(dir_name, epoch + 1))



In [ ]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "test_colab_GPU_highRAM_8_val_8"
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=2e-5,
          validate_per=100,
          run_name=run_name,
          run_descrption="Colab with highRam, lr=2e-5, validate per 100, batch 16")


In [ ]:
gc.collect()
torch.cuda.empty_cache()
main()

In [ ]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "test_colab_GPU_highRAM_8_lr_5e-6"
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=5e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="Colab with highRam, lr=5e-6, validate per 259, batch 8")


In [ ]:
gc.collect()
torch.cuda.empty_cache()
main()

In [ ]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "test_colab_GPU_highRAM_8_lr_5e-6_10epochs"
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=10,
          batch_size=8,
          learning_rate=5e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="Colab with highRam, lr=5e-6, validate per 250, batch 8, 10epochs")


In [ ]:
gc.collect()
torch.cuda.empty_cache()
main()

In [ ]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "test_colab_GPU_highRAM_very__verylow_very_lr_(5e-7)"
    model_name = 'codebert-base'
    checkpoint_location = '/content/drive/MyDrive/CS5814/HW3/codebert_finetune_runs/test_colab_GPU_highRAM_8_lr_5e-6/epoch_5'
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=5e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="Colab with highRam, lr=5e-7, validate per 250, batch 8, 5epochs")


In [ ]:
gc.collect()
torch.cuda.empty_cache()
main()

In [22]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "Softmax + CEL_3e-6 acc 3 good?"
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=3e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="Colab with Softmax + CrossEntropyLoss, lr=3e-6, validate per 250, batch 8, 5 epochs")


In [23]:
torch.cuda.empty_cache()
main()

Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initial

Epoch:0 Step:0 Training_loss:0.633118, Acc_avg:62.50% Training_loss_avg:0.633118
Validating:


52it [00:07,  6.59it/s]


Epoch:0 Step:0 Val_loss:0.673894, Val_Acc_avg:56.94%
Epoch:0 Step:8 Training_loss:0.675437, Acc_avg:68.75% Training_loss_avg:0.654277
Epoch:0 Step:16 Training_loss:0.779569, Acc_avg:58.33% Training_loss_avg:0.696041
Epoch:0 Step:24 Training_loss:0.750523, Acc_avg:53.12% Training_loss_avg:0.709662
Epoch:0 Step:32 Training_loss:0.651663, Acc_avg:57.50% Training_loss_avg:0.698062
Epoch:0 Step:40 Training_loss:0.725335, Acc_avg:54.17% Training_loss_avg:0.702607
Epoch:0 Step:48 Training_loss:0.818100, Acc_avg:50.00% Training_loss_avg:0.719106
Epoch:0 Step:56 Training_loss:0.605680, Acc_avg:53.12% Training_loss_avg:0.704928
Epoch:0 Step:64 Training_loss:0.646780, Acc_avg:55.56% Training_loss_avg:0.698467
Epoch:0 Step:72 Training_loss:0.807646, Acc_avg:52.50% Training_loss_avg:0.709385
Epoch:0 Step:80 Training_loss:0.726775, Acc_avg:51.14% Training_loss_avg:0.710966
Epoch:0 Step:88 Training_loss:0.629971, Acc_avg:53.12% Training_loss_avg:0.704216
Epoch:0 Step:96 Training_loss:0.709214, Acc_av

...

Epoch:4 Step:2728 Val_loss:0.633293, Val_Acc_avg:56.94%
Epoch:4 Step:2736 Training_loss:0.711903, Acc_avg:62.50% Training_loss_avg:0.632935
Epoch:4 Step:2744 Training_loss:0.426640, Acc_avg:62.75% Training_loss_avg:0.628876
Epoch:4 Step:2752 Training_loss:0.441044, Acc_avg:63.25% Training_loss_avg:0.624105
Epoch:4 Step:2760 Training_loss:0.670799, Acc_avg:64.00% Training_loss_avg:0.621994
Epoch:4 Step:2768 Training_loss:0.562493, Acc_avg:63.75% Training_loss_avg:0.620141
Epoch:4 Step:2776 Training_loss:0.639966, Acc_avg:64.00% Training_loss_avg:0.618254
Epoch:4 Step:2784 Training_loss:0.686932, Acc_avg:63.50% Training_loss_avg:0.620906
Epoch:4 Step:2792 Training_loss:0.509133, Acc_avg:63.50% Training_loss_avg:0.618637
Epoch:4 Step:2800 Training_loss:0.998151, Acc_avg:63.25% Training_loss_avg:0.625958
Epoch:4 Step:2808 Training_loss:0.774715, Acc_avg:63.00% Training_loss_avg:0.628046
Epoch:4 Step:2816 Training_loss:0.705545, Acc_avg:62.75% Training_loss_avg:0.628974
Epoch:4 Step:2824 Tr

52it [00:07,  6.61it/s]


Epoch:4 Step:2976 Val_loss:0.630250, Val_Acc_avg:56.94%
Epoch:4 Step:2984 Training_loss:0.640000, Acc_avg:62.75% Training_loss_avg:0.640714
Epoch:4 Step:2992 Training_loss:0.568964, Acc_avg:63.00% Training_loss_avg:0.640887
Epoch:4 Step:3000 Training_loss:0.693123, Acc_avg:63.00% Training_loss_avg:0.641536
Epoch:4 Step:3008 Training_loss:0.769695, Acc_avg:62.50% Training_loss_avg:0.645865
Epoch:4 Step:3016 Training_loss:0.524327, Acc_avg:62.75% Training_loss_avg:0.644458
Epoch:4 Step:3024 Training_loss:0.489376, Acc_avg:64.00% Training_loss_avg:0.639579
Epoch:4 Step:3032 Training_loss:0.615687, Acc_avg:64.50% Training_loss_avg:0.636949
Epoch:4 Step:3040 Training_loss:0.736089, Acc_avg:63.50% Training_loss_avg:0.639798
Epoch:4 Step:3048 Training_loss:0.581482, Acc_avg:64.00% Training_loss_avg:0.637355
Epoch:4 Step:3056 Training_loss:0.683434, Acc_avg:63.50% Training_loss_avg:0.636629
Epoch:4 Step:3064 Training_loss:0.647887, Acc_avg:63.25% Training_loss_avg:0.636883
Epoch:4 Step:3072 Tr

52it [00:07,  6.61it/s]


Epoch:4 Step:3224 Val_loss:0.642710, Val_Acc_avg:52.31%
Epoch:4 Step:3232 Training_loss:0.572043, Acc_avg:63.25% Training_loss_avg:0.629549
Epoch:4 Step:3240 Training_loss:0.609044, Acc_avg:63.25% Training_loss_avg:0.631257
Epoch:4 Step:3248 Training_loss:0.760223, Acc_avg:62.75% Training_loss_avg:0.633554
Epoch:4 Step:3256 Training_loss:0.718417, Acc_avg:62.50% Training_loss_avg:0.635204
Epoch:4 Step:3264 Training_loss:0.851812, Acc_avg:62.00% Training_loss_avg:0.639272
Epoch:4 Step:3272 Training_loss:0.612754, Acc_avg:61.75% Training_loss_avg:0.638601
Epoch:4 Step:3280 Training_loss:0.608137, Acc_avg:61.75% Training_loss_avg:0.637694
Epoch:4 Step:3288 Training_loss:0.757517, Acc_avg:60.75% Training_loss_avg:0.642164
Epoch:4 Step:3296 Training_loss:0.777500, Acc_avg:60.25% Training_loss_avg:0.644037
Epoch:4 Step:3304 Training_loss:0.600789, Acc_avg:60.25% Training_loss_avg:0.643774
Epoch:4 Step:3312 Training_loss:0.655846, Acc_avg:60.25% Training_loss_avg:0.642777
Epoch:4 Step:3320 Tr

In [24]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "Softmax + CEL_5e-6 acc 3 good?"
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=5e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="Colab with Softmax + CrossEntropyLoss, lr=5e-6, validate per 250, batch 8, 5 epochs")


In [25]:
torch.cuda.empty_cache()
main()

Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initial

Epoch:0 Step:0 Training_loss:0.624135, Acc_avg:75.00% Training_loss_avg:0.624135
Validating:


52it [00:07,  6.59it/s]


Epoch:0 Step:0 Val_loss:0.683608, Val_Acc_avg:58.00%
Epoch:0 Step:8 Training_loss:0.650295, Acc_avg:68.75% Training_loss_avg:0.637215
Epoch:0 Step:16 Training_loss:0.696938, Acc_avg:66.67% Training_loss_avg:0.657123
Epoch:0 Step:24 Training_loss:0.658840, Acc_avg:62.50% Training_loss_avg:0.657552
Epoch:0 Step:32 Training_loss:0.643593, Acc_avg:62.50% Training_loss_avg:0.654760
Epoch:0 Step:40 Training_loss:0.615486, Acc_avg:64.58% Training_loss_avg:0.648215
Epoch:0 Step:48 Training_loss:0.602531, Acc_avg:67.86% Training_loss_avg:0.641688
Epoch:0 Step:56 Training_loss:0.658548, Acc_avg:67.19% Training_loss_avg:0.643796
Epoch:0 Step:64 Training_loss:0.693928, Acc_avg:65.28% Training_loss_avg:0.649366
Epoch:0 Step:72 Training_loss:0.572569, Acc_avg:66.25% Training_loss_avg:0.641686
Epoch:0 Step:80 Training_loss:0.602112, Acc_avg:67.05% Training_loss_avg:0.638089
Epoch:0 Step:88 Training_loss:0.542588, Acc_avg:68.75% Training_loss_avg:0.630130
Epoch:0 Step:96 Training_loss:0.542329, Acc_av

...

Epoch:4 Step:2728 Val_loss:0.612235, Val_Acc_avg:54.00%
Epoch:4 Step:2736 Training_loss:0.539447, Acc_avg:65.50% Training_loss_avg:0.608256
Epoch:4 Step:2744 Training_loss:0.529305, Acc_avg:65.25% Training_loss_avg:0.611324
Epoch:4 Step:2752 Training_loss:0.573081, Acc_avg:65.00% Training_loss_avg:0.611113
Epoch:4 Step:2760 Training_loss:0.765467, Acc_avg:64.75% Training_loss_avg:0.613746
Epoch:4 Step:2768 Training_loss:0.564825, Acc_avg:64.75% Training_loss_avg:0.613173
Epoch:4 Step:2776 Training_loss:0.447913, Acc_avg:65.75% Training_loss_avg:0.608033
Epoch:4 Step:2784 Training_loss:0.549314, Acc_avg:66.25% Training_loss_avg:0.607126
Epoch:4 Step:2792 Training_loss:0.623063, Acc_avg:66.00% Training_loss_avg:0.607707
Epoch:4 Step:2800 Training_loss:0.757162, Acc_avg:65.50% Training_loss_avg:0.611219
Epoch:4 Step:2808 Training_loss:0.619737, Acc_avg:65.50% Training_loss_avg:0.611435
Epoch:4 Step:2816 Training_loss:0.600429, Acc_avg:66.25% Training_loss_avg:0.609289
Epoch:4 Step:2824 Tr

52it [00:07,  6.61it/s]


Epoch:4 Step:2976 Val_loss:0.631534, Val_Acc_avg:52.00%
Epoch:4 Step:2984 Training_loss:0.831910, Acc_avg:66.00% Training_loss_avg:0.611208
Epoch:4 Step:2992 Training_loss:0.481054, Acc_avg:65.75% Training_loss_avg:0.612532
Epoch:4 Step:3000 Training_loss:0.420478, Acc_avg:66.00% Training_loss_avg:0.610802
Epoch:4 Step:3008 Training_loss:0.454510, Acc_avg:66.00% Training_loss_avg:0.610176
Epoch:4 Step:3016 Training_loss:0.412283, Acc_avg:66.25% Training_loss_avg:0.608081
Epoch:4 Step:3024 Training_loss:0.612662, Acc_avg:66.00% Training_loss_avg:0.605293
Epoch:4 Step:3032 Training_loss:0.609820, Acc_avg:66.50% Training_loss_avg:0.601040
Epoch:4 Step:3040 Training_loss:0.669935, Acc_avg:66.50% Training_loss_avg:0.600386
Epoch:4 Step:3048 Training_loss:0.818671, Acc_avg:67.00% Training_loss_avg:0.599390
Epoch:4 Step:3056 Training_loss:0.457971, Acc_avg:68.00% Training_loss_avg:0.593418
Epoch:4 Step:3064 Training_loss:0.621092, Acc_avg:68.00% Training_loss_avg:0.593940
Epoch:4 Step:3072 Tr

52it [00:07,  6.61it/s]


Epoch:4 Step:3224 Val_loss:0.616687, Val_Acc_avg:54.00%
Epoch:4 Step:3232 Training_loss:0.479017, Acc_avg:68.00% Training_loss_avg:0.588731
Epoch:4 Step:3240 Training_loss:0.649384, Acc_avg:67.75% Training_loss_avg:0.591542
Epoch:4 Step:3248 Training_loss:0.830330, Acc_avg:66.50% Training_loss_avg:0.596544
Epoch:4 Step:3256 Training_loss:0.769595, Acc_avg:65.75% Training_loss_avg:0.598551
Epoch:4 Step:3264 Training_loss:0.555436, Acc_avg:66.00% Training_loss_avg:0.596228
Epoch:4 Step:3272 Training_loss:0.592162, Acc_avg:65.75% Training_loss_avg:0.596068
Epoch:4 Step:3280 Training_loss:0.385554, Acc_avg:66.00% Training_loss_avg:0.591919
Epoch:4 Step:3288 Training_loss:0.606692, Acc_avg:66.75% Training_loss_avg:0.589327
Epoch:4 Step:3296 Training_loss:0.425845, Acc_avg:66.50% Training_loss_avg:0.588118
Epoch:4 Step:3304 Training_loss:0.583959, Acc_avg:66.50% Training_loss_avg:0.588773
Epoch:4 Step:3312 Training_loss:0.590704, Acc_avg:66.25% Training_loss_avg:0.589352
Epoch:4 Step:3320 Tr

In [179]:
print(dir_name)

codebert_finetune_runs/lr_5e-5 redo, 512 split with attention! custom model redo 3


In [64]:
def split_loader(run_dir):
    data_type = ['train', 'val', 'test']
    data_split_type = ['X', 'A', 'Y']

    split_list = []

    for data_type_id in data_type:
      for split_type in data_split_type:
          with open('{}/{}_{}.pickle'.format(run_dir,data_type_id, split_type), 'rb') as input_file:
            object_file = pickle.load(input_file)
          split_list.append(object_file)
    X_train, A_train, Y_train, X_val, A_val, Y_val, X_test, A_test, Y_test = split_list

    return (X_train, A_train, Y_train), (X_val, A_val, Y_val), (X_test, A_test, Y_test)

In [173]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """
    run_name = "lr_5e-6 redo, 512 split with attention! redo5"
    run_dir = "codebert_finetune_runs/{}".format(run_name)
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False
    load_splits = False
    save_data = True

    if load_splits:
      train_data, val_data, test_data = split_loader(run_dir)
      X_train, A_train, Y_train = train_data
      X_val, A_val, Y_val = val_data
      X_test, A_test, Y_test = test_data
      print("loaded data splits")

    else:
      print("generating data splits")

      code_df = preprocess_data(file_loc='code_dataset.jsonl')
      train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

      X_train, A_train, Y_train = train_data
      X_val, A_val, Y_val = val_data
      X_test, A_test, Y_test = test_data

      data_type = ['train', 'val', 'test']
      data_split_type = ['X', 'A', 'Y']

      print(X_train.shape)
    # Creating dir to save logs and checkpoints, re
    dir_name = "{}".format(run_dir)
    if os.path.exists(dir_name):
        input("run name already exists, press Enter to overwrite")
    else:
        os.makedirs(dir_name)


    if save_data:
      print("saving data splits")

      data_all = [train_data, val_data, test_data]
      for i, data in enumerate(data_all):
        for j, split in enumerate(data):
          with open('{}/{}_{}.pickle'.format(run_dir,data_type[i], data_split_type[j]), 'wb') as handle:
            pickle.dump(split, handle)


    print(train_data[0].shape)
    print("Data points: {}".format(len(train_data)))

    # Loading model from checkpoint if location provided
    if online:
        print("loading model from online")
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        print("loading model from local repo")

        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        print("loading model from checkpoint: {}".format(checkpoint_location))
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=5e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="lr_5e-6 redo, 256 split, saving splits")


In [174]:
torch.cuda.empty_cache()
main()

generating data splits
Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125
Data points: 8000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


torch.Size([3331, 512])
saving data splits
torch.Size([3331, 512])
Data points: 3
loading model from local repo


Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for p

Epoch:0 Step:0 Training_loss:0.717449, Acc_avg:25.00% Training_loss_avg:0.717449
Validating:


...

Epoch:4 Step:2728 Val_loss:0.649943, Val_Acc_avg:58.75%
Epoch:4 Step:2736 Training_loss:0.263467, Acc_avg:68.00% Training_loss_avg:0.586400
Epoch:4 Step:2744 Training_loss:0.700553, Acc_avg:67.75% Training_loss_avg:0.587623
Epoch:4 Step:2752 Training_loss:0.393567, Acc_avg:68.75% Training_loss_avg:0.576000
Epoch:4 Step:2760 Training_loss:0.514447, Acc_avg:68.75% Training_loss_avg:0.572561
Epoch:4 Step:2768 Training_loss:0.885084, Acc_avg:68.50% Training_loss_avg:0.571755
Epoch:4 Step:2776 Training_loss:0.334351, Acc_avg:69.25% Training_loss_avg:0.562738
Epoch:4 Step:2784 Training_loss:0.365829, Acc_avg:70.25% Training_loss_avg:0.556063
Epoch:4 Step:2792 Training_loss:0.517371, Acc_avg:69.50% Training_loss_avg:0.558753
Epoch:4 Step:2800 Training_loss:0.607387, Acc_avg:69.75% Training_loss_avg:0.557618
Epoch:4 Step:2808 Training_loss:0.594127, Acc_avg:69.75% Training_loss_avg:0.559957
Epoch:4 Step:2816 Training_loss:0.508931, Acc_avg:70.00% Training_loss_avg:0.556462
Epoch:4 Step:2824 Tr

52it [00:07,  6.59it/s]


Epoch:4 Step:2976 Val_loss:0.661207, Val_Acc_avg:63.25%
Epoch:4 Step:2984 Training_loss:0.743741, Acc_avg:68.25% Training_loss_avg:0.574354
Epoch:4 Step:2992 Training_loss:0.401419, Acc_avg:68.50% Training_loss_avg:0.573041
Epoch:4 Step:3000 Training_loss:0.638137, Acc_avg:69.00% Training_loss_avg:0.574354
Epoch:4 Step:3008 Training_loss:0.469935, Acc_avg:69.25% Training_loss_avg:0.574492
Epoch:4 Step:3016 Training_loss:0.629365, Acc_avg:69.00% Training_loss_avg:0.572833
Epoch:4 Step:3024 Training_loss:0.473024, Acc_avg:69.00% Training_loss_avg:0.574031
Epoch:4 Step:3032 Training_loss:0.455103, Acc_avg:69.25% Training_loss_avg:0.568032
Epoch:4 Step:3040 Training_loss:0.512412, Acc_avg:69.75% Training_loss_avg:0.563583
Epoch:4 Step:3048 Training_loss:0.599136, Acc_avg:70.25% Training_loss_avg:0.561735
Epoch:4 Step:3056 Training_loss:0.573720, Acc_avg:70.75% Training_loss_avg:0.559576
Epoch:4 Step:3064 Training_loss:0.361666, Acc_avg:71.25% Training_loss_avg:0.554976
Epoch:4 Step:3072 Tr

52it [00:07,  6.60it/s]


Epoch:4 Step:3224 Val_loss:0.680723, Val_Acc_avg:62.50%
Epoch:4 Step:3232 Training_loss:0.470278, Acc_avg:72.75% Training_loss_avg:0.533979
Epoch:4 Step:3240 Training_loss:0.667614, Acc_avg:72.00% Training_loss_avg:0.538905
Epoch:4 Step:3248 Training_loss:0.584908, Acc_avg:72.25% Training_loss_avg:0.540639
Epoch:4 Step:3256 Training_loss:0.419002, Acc_avg:72.25% Training_loss_avg:0.541701
Epoch:4 Step:3264 Training_loss:0.601236, Acc_avg:72.00% Training_loss_avg:0.543475
Epoch:4 Step:3272 Training_loss:0.678155, Acc_avg:72.00% Training_loss_avg:0.546434
Epoch:4 Step:3280 Training_loss:0.534536, Acc_avg:71.75% Training_loss_avg:0.544336
Epoch:4 Step:3288 Training_loss:0.441876, Acc_avg:71.75% Training_loss_avg:0.542117
Epoch:4 Step:3296 Training_loss:0.364218, Acc_avg:72.75% Training_loss_avg:0.536356
Epoch:4 Step:3304 Training_loss:0.443095, Acc_avg:73.25% Training_loss_avg:0.531622
Epoch:4 Step:3312 Training_loss:0.235256, Acc_avg:73.50% Training_loss_avg:0.529201
Epoch:4 Step:3320 Tr

In [177]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """
    run_name = "lr_5e-6 redo, 512 split with attention! redo 6"
    run_dir = "codebert_finetune_runs/{}".format(run_name)
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False
    load_splits = False
    save_data = True

    if load_splits:
      train_data, val_data, test_data = split_loader(run_dir)
      X_train, A_train, Y_train = train_data
      X_val, A_val, Y_val = val_data
      X_test, A_test, Y_test = test_data
      print("loaded data splits")

    else:
      print("generating data splits")

      code_df = preprocess_data(file_loc='code_dataset.jsonl')
      train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

      X_train, A_train, Y_train = train_data
      X_val, A_val, Y_val = val_data
      X_test, A_test, Y_test = test_data

      data_type = ['train', 'val', 'test']
      data_split_type = ['X', 'A', 'Y']

      print(X_train.shape)
    # Creating dir to save logs and checkpoints, re
    dir_name = "{}".format(run_dir)
    if os.path.exists(dir_name):
        input("run name already exists, press Enter to overwrite")
    else:
        os.makedirs(dir_name)


    if save_data:
      print("saving data splits")

      data_all = [train_data, val_data, test_data]
      for i, data in enumerate(data_all):
        for j, split in enumerate(data):
          with open('{}/{}_{}.pickle'.format(run_dir,data_type[i], data_split_type[j]), 'wb') as handle:
            pickle.dump(split, handle)


    print(train_data[0].shape)
    print("Data points: {}".format(len(train_data)))

    # Loading model from checkpoint if location provided
    if online:
        print("loading model from online")
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        print("loading model from local repo")

        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        print("loading model from checkpoint: {}".format(checkpoint_location))
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=3e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="lr_3e-6 redo, 256 split, saving splits")


In [178]:
torch.cuda.empty_cache()
main()

generating data splits
Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125
Data points: 8000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


torch.Size([3331, 512])
saving data splits
torch.Size([3331, 512])
Data points: 3
loading model from local repo


Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for p

Epoch:0 Step:0 Training_loss:0.667126, Acc_avg:62.50% Training_loss_avg:0.667126
Validating:


...

Epoch:4 Step:2728 Val_loss:0.607856, Val_Acc_avg:66.00%
Epoch:4 Step:2736 Training_loss:0.583592, Acc_avg:69.75% Training_loss_avg:0.566632
Epoch:4 Step:2744 Training_loss:0.831829, Acc_avg:69.50% Training_loss_avg:0.570145
Epoch:4 Step:2752 Training_loss:0.421502, Acc_avg:69.25% Training_loss_avg:0.571403
Epoch:4 Step:2760 Training_loss:0.699395, Acc_avg:69.50% Training_loss_avg:0.570339
Epoch:4 Step:2768 Training_loss:0.506170, Acc_avg:70.00% Training_loss_avg:0.568305
Epoch:4 Step:2776 Training_loss:0.426973, Acc_avg:69.50% Training_loss_avg:0.569998
Epoch:4 Step:2784 Training_loss:0.293821, Acc_avg:70.00% Training_loss_avg:0.567755
Epoch:4 Step:2792 Training_loss:0.795251, Acc_avg:70.25% Training_loss_avg:0.569779
Epoch:4 Step:2800 Training_loss:0.606037, Acc_avg:69.75% Training_loss_avg:0.570472
Epoch:4 Step:2808 Training_loss:0.929869, Acc_avg:69.25% Training_loss_avg:0.575560
Epoch:4 Step:2816 Training_loss:0.326508, Acc_avg:69.75% Training_loss_avg:0.571380
Epoch:4 Step:2824 Tr

52it [00:07,  6.61it/s]


Epoch:4 Step:2976 Val_loss:0.596453, Val_Acc_avg:67.25%
Epoch:4 Step:2984 Training_loss:0.546762, Acc_avg:72.50% Training_loss_avg:0.546559
Epoch:4 Step:2992 Training_loss:0.468825, Acc_avg:72.00% Training_loss_avg:0.550518
Epoch:4 Step:3000 Training_loss:0.762902, Acc_avg:71.25% Training_loss_avg:0.558095
Epoch:4 Step:3008 Training_loss:0.446309, Acc_avg:71.75% Training_loss_avg:0.554606
Epoch:4 Step:3016 Training_loss:0.682584, Acc_avg:71.25% Training_loss_avg:0.555735
Epoch:4 Step:3024 Training_loss:0.399476, Acc_avg:71.00% Training_loss_avg:0.557200
Epoch:4 Step:3032 Training_loss:0.581398, Acc_avg:71.00% Training_loss_avg:0.556544
Epoch:4 Step:3040 Training_loss:0.401827, Acc_avg:71.00% Training_loss_avg:0.556771
Epoch:4 Step:3048 Training_loss:0.803543, Acc_avg:70.75% Training_loss_avg:0.566597
Epoch:4 Step:3056 Training_loss:0.820118, Acc_avg:69.75% Training_loss_avg:0.575939
Epoch:4 Step:3064 Training_loss:0.607218, Acc_avg:70.25% Training_loss_avg:0.575142
Epoch:4 Step:3072 Tr

52it [00:07,  6.60it/s]


Epoch:4 Step:3224 Val_loss:0.613616, Val_Acc_avg:63.75%
Epoch:4 Step:3232 Training_loss:0.580986, Acc_avg:69.00% Training_loss_avg:0.578269
Epoch:4 Step:3240 Training_loss:0.838321, Acc_avg:68.50% Training_loss_avg:0.582393
Epoch:4 Step:3248 Training_loss:0.584132, Acc_avg:68.50% Training_loss_avg:0.584188
Epoch:4 Step:3256 Training_loss:0.773404, Acc_avg:67.50% Training_loss_avg:0.593249
Epoch:4 Step:3264 Training_loss:0.456495, Acc_avg:67.50% Training_loss_avg:0.595212
Epoch:4 Step:3272 Training_loss:0.729953, Acc_avg:67.00% Training_loss_avg:0.601798
Epoch:4 Step:3280 Training_loss:0.616288, Acc_avg:66.50% Training_loss_avg:0.603514
Epoch:4 Step:3288 Training_loss:0.567872, Acc_avg:66.50% Training_loss_avg:0.602931
Epoch:4 Step:3296 Training_loss:0.597734, Acc_avg:66.25% Training_loss_avg:0.601283
Epoch:4 Step:3304 Training_loss:0.517985, Acc_avg:66.50% Training_loss_avg:0.602033
Epoch:4 Step:3312 Training_loss:0.518588, Acc_avg:66.75% Training_loss_avg:0.597189
Epoch:4 Step:3320 Tr

In [ ]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """
    run_name = "lr_5e-6 redo, 512 split with attention!"
    run_dir = "codebert_finetune_runs/{}".format(run_name)
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False
    load_splits = False
    save_data = True

    if load_splits:
      train_data, val_data, test_data = split_loader(run_dir)
      X_train, A_train, Y_train = train_data
      X_val, A_val, Y_val = val_data
      X_test, A_test, Y_test = test_data
      print("loaded data splits")

    else:
      print("generating data splits")

      code_df = preprocess_data(file_loc='code_dataset.jsonl')
      train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

      X_train, A_train, Y_train = train_data
      X_val, A_val, Y_val = val_data
      X_test, A_test, Y_test = test_data

      data_type = ['train', 'val', 'test']
      data_split_type = ['X', 'A', 'Y']

    
    # Creating dir to save logs and checkpoints, re
    dir_name = "{}".format(run_dir)
    if os.path.exists(dir_name):
        input("run name already exists, press Enter to overwrite")
    else:
        os.makedirs(dir_name)


    if save_data:
      print("saving data splits")

      data_all = [train_data, val_data, test_data]
      for i, data in enumerate(data_all):
        for j, split in enumerate(data):
          with open('{}/{}_{}.pickle'.format(run_dir,data_type[i], data_split_type[j]), 'wb') as handle:
            pickle.dump(split, handle)


    print(train_data[0].shape)
    print("Data points: {}".format(len(train_data)))

    # Loading model from checkpoint if location provided
    if online:
        print("loading model from online")
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        print("loading model from local repo")

        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        print("loading model from checkpoint: {}".format(checkpoint_location))
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=5e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="lr_5e-6 redo, 512 split, saving splits")


In [78]:
torch.cuda.empty_cache()
main()

generating data splits
Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125
Data points: 8000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


saving data splits
torch.Size([3331, 512])
Data points: 3
loading model from local repo


Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for p

run name already exists, exiting to prevent overwriting


...

Epoch:4 Step:2728 Val_loss:0.613492, Val_Acc_avg:62.25%
Epoch:4 Step:2736 Training_loss:0.675986, Acc_avg:64.75% Training_loss_avg:0.584266
Epoch:4 Step:2744 Training_loss:0.490294, Acc_avg:64.75% Training_loss_avg:0.583478
Epoch:4 Step:2752 Training_loss:0.333614, Acc_avg:65.25% Training_loss_avg:0.579448
Epoch:4 Step:2760 Training_loss:0.540486, Acc_avg:65.50% Training_loss_avg:0.577009
Epoch:4 Step:2768 Training_loss:0.316952, Acc_avg:66.50% Training_loss_avg:0.571156
Epoch:4 Step:2776 Training_loss:0.438694, Acc_avg:66.50% Training_loss_avg:0.571164
Epoch:4 Step:2784 Training_loss:0.465745, Acc_avg:66.50% Training_loss_avg:0.571224
Epoch:4 Step:2792 Training_loss:0.420821, Acc_avg:67.25% Training_loss_avg:0.564551
Epoch:4 Step:2800 Training_loss:0.633117, Acc_avg:67.75% Training_loss_avg:0.563012
Epoch:4 Step:2808 Training_loss:0.894070, Acc_avg:66.75% Training_loss_avg:0.570994
Epoch:4 Step:2816 Training_loss:0.432145, Acc_avg:67.50% Training_loss_avg:0.566419
Epoch:4 Step:2824 Tr

52it [00:07,  6.61it/s]


Epoch:4 Step:2976 Val_loss:0.627388, Val_Acc_avg:61.75%
Epoch:4 Step:2984 Training_loss:0.679367, Acc_avg:70.50% Training_loss_avg:0.575101
Epoch:4 Step:2992 Training_loss:0.418711, Acc_avg:70.50% Training_loss_avg:0.570775
Epoch:4 Step:3000 Training_loss:0.326063, Acc_avg:71.75% Training_loss_avg:0.564331
Epoch:4 Step:3008 Training_loss:0.496098, Acc_avg:71.75% Training_loss_avg:0.563876
Epoch:4 Step:3016 Training_loss:0.577549, Acc_avg:72.00% Training_loss_avg:0.565962
Epoch:4 Step:3024 Training_loss:0.836364, Acc_avg:71.50% Training_loss_avg:0.572310
Epoch:4 Step:3032 Training_loss:0.520748, Acc_avg:72.25% Training_loss_avg:0.568741
Epoch:4 Step:3040 Training_loss:0.412200, Acc_avg:72.25% Training_loss_avg:0.566795
Epoch:4 Step:3048 Training_loss:0.821325, Acc_avg:71.50% Training_loss_avg:0.571175
Epoch:4 Step:3056 Training_loss:0.576077, Acc_avg:71.00% Training_loss_avg:0.572529
Epoch:4 Step:3064 Training_loss:0.509429, Acc_avg:71.50% Training_loss_avg:0.571286
Epoch:4 Step:3072 Tr

52it [00:07,  6.62it/s]


Epoch:4 Step:3224 Val_loss:0.642445, Val_Acc_avg:60.00%
Epoch:4 Step:3232 Training_loss:0.868251, Acc_avg:64.75% Training_loss_avg:0.619867
Epoch:4 Step:3240 Training_loss:0.633377, Acc_avg:65.00% Training_loss_avg:0.617536
Epoch:4 Step:3248 Training_loss:0.658400, Acc_avg:64.75% Training_loss_avg:0.618990
Epoch:4 Step:3256 Training_loss:0.483797, Acc_avg:64.75% Training_loss_avg:0.611503
Epoch:4 Step:3264 Training_loss:0.455788, Acc_avg:65.50% Training_loss_avg:0.610233
Epoch:4 Step:3272 Training_loss:0.694596, Acc_avg:64.75% Training_loss_avg:0.616390
Epoch:4 Step:3280 Training_loss:0.668522, Acc_avg:64.50% Training_loss_avg:0.619533
Epoch:4 Step:3288 Training_loss:0.491824, Acc_avg:65.00% Training_loss_avg:0.615889
Epoch:4 Step:3296 Training_loss:0.522814, Acc_avg:65.00% Training_loss_avg:0.613904
Epoch:4 Step:3304 Training_loss:0.501287, Acc_avg:65.75% Training_loss_avg:0.611481
Epoch:4 Step:3312 Training_loss:0.601969, Acc_avg:65.25% Training_loss_avg:0.614005
Epoch:4 Step:3320 Tr

In [119]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(512,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      cls_hs = self.bert(sent_id, attention_mask=mask)

      x = self.fc1(cls_hs.logits)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [150]:
"""
Main configuration function for a given finetune run
:return: None
"""
run_name = "lr_5e-5 redo, 512 split with attention! custom model redo 3"
run_dir = "codebert_finetune_runs/{}".format(run_name)
model_name = 'codebert-base'
checkpoint_location = None
online = False
load_splits = False
save_data = True

print("generating data splits")

code_df = preprocess_data(file_loc='code_dataset.jsonl')
train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

X_train, A_train, Y_train = train_data
X_val, A_val, Y_val = val_data
X_test, A_test, Y_test = test_data

data_type = ['train', 'val', 'test']
data_split_type = ['X', 'A', 'Y']


# Creating dir to save logs and checkpoints, re
dir_name = "{}".format(run_dir)
if os.path.exists(dir_name):
  input("run name already exists, press Enter to overwrite")
else:
  os.makedirs(dir_name)

if save_data:
  print("saving data splits")

  data_all = [train_data, val_data, test_data]
  for i, data in enumerate(data_all):
    for j, split in enumerate(data):
      with open('{}/{}_{}.pickle'.format(run_dir,data_type[i], data_split_type[j]), 'wb') as handle:
        pickle.dump(split, handle)

# Loading model from checkpoint if location provided
if online:
  print("loading model from online")
  model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
elif checkpoint_location is None:
  print("loading model from local repo")

  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=512)
else:
  print("loading model from checkpoint: {}".format(checkpoint_location))
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)






generating data splits
Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125
Data points: 8000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


saving data splits
loading model from local repo


Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [151]:
# freeze all the parameters
for param in model.parameters():
    param.requires_grad = False

custom_model = BERT_Arch(model)

In [152]:
custom_model = custom_model.to(device)

In [153]:

def train_custom(model, train_data, val_data, epochs=5, batch_size=16, learning_rate=2e-5, validate_per=500,
          run_name="temp", run_descrption=None):
    """
    Main fine-tuning training loop for the provided model

    :param model: model loaded with predefined weights
    :param train_data: tuple of X_train, A_train, Y_train (X = inputs, A = attention, Y = target)
    :param val_data: tuple X_val, A_val, Y_val
    :param epochs: number of epochs for training
    :param batch_size: batch size (see note below about batch_hack)
    :param learning_rate: optimizer learning rate
    :param validate_per: number of weight updates before validation occurs
                            (notes: - if batch_size = 32, and validate_per = 32, validation will occur every batch
                                    - this is wrt the start of each epoch
                                    - validation will always occour at the start of each epoch (step 0))
    :param run_name: name used to saving checkpoints and log files within codebert_finetune_runs
    :param run_descrption: string that is saved to info.txt describing the run


    :return: None (models are saved in checkpoints along with log data)
    """


    # Saving run description.txt
    if run_descrption is not None:
        with open("{}/info.txt".format(dir_name), "a+") as f:
            f.write(run_descrption)

    # Unpacking data
    X_train, A_train, Y_train = train_data
    X_val, A_val, Y_val = val_data


    batch_hack = batch_size  # See note below regarding limited GPU memory

    # Initializing arrays for tracking loss
    train_loss_hist = []
    val_loss_hist = []
    train_pred_hist = []
    # Counter to track batches (see note below related to GPU memory)
    batch_count = 0
    # validate_per_batch = int(validate_per/batch_hack)

    # Moving model to GPU if configured
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    validate_per = int(validate_per/batch_size)

    cross_entropy  = nn.NLLLoss() 
    for epoch in range(epochs):

        # Generating random index for manual shuffling of data each epoch as note using DataLoaders
        permutation = torch.randperm(X_train.shape[0])

        # Note here that only a single element is loaded at each iteration (batch size = 1) due to GPU memory constraint
        for batch_id, i in enumerate(range(0, X_train.shape[0], batch_hack)):

            # Loading batch and moving to device
            indices = permutation[i:i + batch_hack]
            batch_X, batch_Y, batch_A = X_train[indices].to(device), Y_train[indices].to(device), A_train[indices].to(device)


            batch_Y_one_hot = torch.nn.functional.one_hot(batch_Y, num_classes= 2)

            model.train()

            # Forward pass
            #outputs = model(batch_X,labels=batch_Y, attention_mask=batch_A)
            outputs = model(batch_X, batch_A)

            #loss = criterion(loss_clsf.float(), batch_Y_one_hot.float())
            #loss = outputs.loss

            loss = cross_entropy(outputs, batch_Y)


            # Clip params
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


            # Backward pass
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_clsf = nn.Softmax(dim=1)(outputs)
            acc = np.average(torch.eq(batch_Y.cpu(), loss_clsf.argmax(axis=1).cpu()))
            #rint(correct)

            # Tracking loss
            train_loss_hist.append(float(loss.item()))
            train_pred_hist.append(acc)

            # Training output
            train_output = "Epoch:{} Step:{} Training_loss:{:.6f}, Acc_avg:{:.2f}%".format(epoch, i, loss.item(), np.sum(100*train_pred_hist[-50:])/min(len(train_pred_hist), 50))
            print(train_output+" Training_loss_avg:{:.6f}".format(np.average(train_loss_hist[-50:])))
            with open("{}/train_loss.txt".format(dir_name), "a+") as f:
                f.write(train_output+"\n")

            # Validation
            if batch_id % validate_per == 0:
                val_loss_total = 0
                model.eval()
                print("Validating:")
                val_acc = []
                for val_badtch_id, j in tqdm(enumerate(range(0, X_val.shape[0], batch_hack))):
                    # Loading singular validation data (overwrites train data as can only load 1 intp GPU)
                    batch_X, batch_Y, batch_A = X_val[j:j+batch_hack].to(device), Y_val[j:j+batch_hack].to(device), A_val[j:j+batch_hack].to(device)

                    with torch.no_grad():
                        outputs = model(batch_X, batch_A)

                    val_loss = cross_entropy(outputs, batch_Y)
                    val_loss_total += float(val_loss)

                    
                    val_clsf = nn.Softmax(dim=1)(outputs)
                    val_acc.append(np.average(torch.eq(batch_Y.cpu(), val_clsf.argmax(axis=1).cpu())))

                    del batch_X
                    del batch_Y

                # Adding average loss to tracker
                val_average = val_loss_total / (val_badtch_id+1)
                val_loss_hist.append(val_average)

                # Validation output and logging
                val_output = "Epoch:{} Step:{} Val_loss:{:.6f}, Val_Acc_avg:{:.2f}%".format(epoch, i, val_average, np.sum(100*val_acc[-50:])/min(len(val_acc), 50))
                print(val_output)
                with open("{}/val_los.txt".format(dir_name), "a+") as f:
                    f.write(val_output+"\n")

        # End of epoch checkpoint
        #model.save_pretrained("{}/epoch_{}".format(dir_name, epoch + 1))
        torch.save(model, "{}/epoch_{}".format(dir_name, epoch + 1))


In [ ]:
train_custom(model=custom_model,
      train_data=train_data,
      val_data=val_data,
      epochs=10,
      batch_size=8,
      learning_rate=5e-5,
      validate_per=250,
      run_name=run_name,
      run_descrption="lr_5e-5 redo")

# New Section

# New Section